In [ ]:
# Ячейка 1: Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('../src')

from data_loader import DataLoader
from statistical_analyzer import StatisticalAnalyzer

# Настройка визуализации
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
# Ячейка 2: Загрузка подготовленных данных
print("📊 ЗАГРУЗКА ПОДГОТОВЛЕННЫХ ДАННЫХ...")

df = pd.read_csv('../data/processed/cleaned_marketing_ab.csv')
print(f"Размер данных: {df.shape}")
print(f"Колонки: {list(df.columns)}")
print(f"\nПервые 3 строки:")
print(df.head(3))

In [ ]:
# Ячейка 3: Инициализация анализатора
analyzer = StatisticalAnalyzer(df)

In [ ]:
# Ячейка 4: SRM проверка (Sample Ratio Mismatch)
print("🔍 SRM ПРОВЕРКА (Sample Ratio Mismatch)")
srm_results = analyzer.validate_srm()

print(f"Результат: {srm_results['message']}")
print(f"P-value: {srm_results['p_value']:.6f}")
print("\nРаспределение по группам:")
for group, count in srm_results['group_counts'].items():
    percentage = count / sum(srm_results['group_counts'].values()) * 100
    print(f"  {group}: {count} ({percentage:.1f}%)")

In [ ]:
# Ячейка 5: Расчет конверсии по группам
print("🎯 РАСЧЕТ КОНВЕРСИИ ПО ГРУППАМ")
conversion_rates = analyzer.calculate_conversion_rates()

for group, metrics in conversion_rates.items():
    if group in ['ad', 'psa']:
        print(f"\n{group.upper()} группа:")
        print(f"  Конверсий: {metrics['conversions']}/{metrics['total']}")
        print(f"  Конверсия: {metrics['conversion_percentage']:.3f}%")
        print(f"  Rate: {metrics['conversion_rate']:.4f}")

if 'difference' in conversion_rates:
    diff = conversion_rates['difference']
    print(f"\n📈 РАЗНИЦА:")
    print(f"  Абсолютная: {diff['absolute']:.4f}")
    print(f"  Относительная: {diff['relative']:.2f}%")

In [ ]:
# Ячейка 6: Визуализация конверсии
plt.figure(figsize=(10, 6))

# Данные для графика
groups = ['psa', 'ad']
rates = [conversion_rates[g]['conversion_rate'] for g in groups]
conversions = [conversion_rates[g]['conversions'] for g in groups]
totals = [conversion_rates[g]['total'] for g in groups]

bars = plt.bar(groups, rates, color=['lightcoral', 'lightblue'], alpha=0.8)

plt.title('Конверсия по группам A/B теста', fontsize=14, fontweight='bold')
plt.ylabel('Conversion Rate', fontsize=12)
plt.ylim(0, max(rates) * 1.2)

# Добавляем значения на столбцы
for bar, rate, conv, total in zip(bars, rates, conversions, totals):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.001,
             f'{rate:.3%}\n({conv}/{total})',
             ha='center', va='bottom', fontweight='bold')

plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Ячейка 7: Статистическая значимость (Z-тест)
print("📊 ПРОВЕРКА СТАТИСТИЧЕСКОЙ ЗНАЧИМОСТИ")
prop_test = analyzer.perform_proportion_test()

print(f"Z-статистика: {prop_test['z_statistic']:.4f}")
print(f"P-value: {prop_test['p_value']:.6f}")
print(f"Статистически значимо: {'✅ ДА' if prop_test['significant'] else '❌ НЕТ'}")

print(f"\nДоверительные интервалы (95%):")
print(f"  AD: [{prop_test['ad_confidence_interval'][0]:.4f}, {prop_test['ad_confidence_interval'][1]:.4f}]")
print(f"  PSA: [{prop_test['psa_confidence_interval'][0]:.4f}, {prop_test['psa_confidence_interval'][1]:.4f}]")
print(f"  Разница: [{prop_test['difference_confidence_interval'][0]:.4f}, {prop_test['difference_confidence_interval'][1]:.4f}]")

In [ ]:
# Ячейка 8: Визуализация доверительных интервалов
plt.figure(figsize=(12, 6))

# Доверительные интервалы для конверсии
plt.subplot(1, 2, 1)
for i, group in enumerate(['psa', 'ad']):
    ci = prop_test[f'{group.lower()}_confidence_interval']
    rate = conversion_rates[group]['conversion_rate']
    
    plt.errorbar(i, rate, yerr=[[rate - ci[0]], [ci[1] - rate]], 
                 fmt='o', capsize=10, capthick=2, markersize=8,
                 label=f'{group.upper()} group')
    
plt.xticks([0, 1], ['PSA', 'AD'])
plt.ylabel('Conversion Rate')
plt.title('Доверительные интервалы конверсии (95%)')
plt.legend()
plt.grid(alpha=0.3)

# Доверительный интервал для разницы
plt.subplot(1, 2, 2)
diff = conversion_rates['difference']['absolute']
diff_ci = prop_test['difference_confidence_interval']

plt.errorbar(0, diff, yerr=[[diff - diff_ci[0]], [diff_ci[1] - diff]], 
             fmt='o', capsize=10, capthick=2, markersize=8, color='green')
plt.axhline(y=0, color='red', linestyle='--', alpha=0.7)
plt.xticks([0], ['Difference (AD - PSA)'])
plt.ylabel('Absolute Difference')
plt.title('Доверительный интервал разницы (95%)')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Ячейка 9: Bootstrap анализ
print("🔄 BOOTSTRAP АНАЛИЗ")
bootstrap_results = analyzer.bootstrap_analysis(n_bootstrap=5000)

print(f"Средняя разница: {bootstrap_results['mean_difference']:.6f}")
print(f"95% ДИ: [{bootstrap_results['confidence_interval_95'][0]:.6f}, {bootstrap_results['confidence_interval_95'][1]:.6f}]")
print(f"90% ДИ: [{bootstrap_results['confidence_interval_90'][0]:.6f}, {bootstrap_results['confidence_interval_90'][1]:.6f}]")
print(f"Bootstrap p-value: {bootstrap_results['p_value']:.6f}")

In [ ]:
# Ячейка 10: Визуализация bootstrap распределения
plt.figure(figsize=(10, 6))

plt.hist(bootstrap_results['bootstrap_differences'], bins=50, 
         alpha=0.7, color='lightblue', edgecolor='black')
plt.axvline(bootstrap_results['mean_difference'], color='red', 
            linestyle='--', linewidth=2, label='Средняя разница')
plt.axvline(0, color='black', linestyle='-', linewidth=1, alpha=0.7)

# Доверительные интервалы
plt.axvline(bootstrap_results['confidence_interval_95'][0], color='orange', 
            linestyle='--', alpha=0.7, label='95% ДИ')
plt.axvline(bootstrap_results['confidence_interval_95'][1], color='orange', 
            linestyle='--', alpha=0.7)

plt.xlabel('Разница в конверсии (AD - PSA)')
plt.ylabel('Frequency')
plt.title('Bootstrap распределение разницы конверсий')
plt.legend()
plt.grid(alpha=0.3)
plt.show()